In [1]:
import os
import numpy as np
import music21 as m21
import pandas as pd
import json
import matplotlib.pyplot as plt
from scipy import stats
from scipy import spatial
import time
import math
from IPython.display import display
from collections import Counter

np.random.seed(777)

## Functions

In [2]:
DIV_CONST = 4

In [3]:
def getSongKey(song):
    key = song.analyze("key")
    return key

In [4]:
def getSongKeyFromMelody_W_Times(melody_w_times_in_k):
    sc_test = m21.stream.Score()
    p0_test = m21.stream.Part()
    p0_test.id = 'part0'
    for pitch_i in melody_w_times_in_k:
        n_i = m21.note.Note(pitch_i[4])
        p0_test.append(n_i)
    sc_test.insert(0, p0_test)
    return getSongKey(sc_test)

In [5]:
# Function to retrieve a list of midi pitch events and its timestamp
def getMelodyDeltaTimes(eventsintrack):
    
    # Initialize array
    DeltaTimes = []
    
    # Initialize cumulative sum
    cum_sum = 0
    
    # Initialize variable to track the time delta
    prev_deltatime = 0
    
    # Traverse the events
    for ev in eventsintrack:
        
        # If a note starts
        if (ev.isNoteOn()):
            
            # Get the pitch name and save it with the cumulative sum, midi pitch and name
            pitch_in_time = m21.pitch.Pitch(ev.pitch)
            DeltaTimes.append((cum_sum, prev_deltatime, pitch_in_time.midi, pitch_in_time.spanish, pitch_in_time))
            
            # Restart the delta time
            prev_deltatime = 0
        
        # Else if there is a delta time
        elif(str(ev.type) == "DeltaTime"):
            
            # We sum the time
            cum_sum += ev.time
            
            # We sum it to the current delta time
            prev_deltatime += ev.time
    
    # Return the array
    return DeltaTimes

In [6]:
def get_SCLM_v100(melody_w_times_A, melody_w_times_B):
    
    # We use a Dynamic Programming approach
    max_len = max(len(melody_w_times_A), len(melody_w_times_B)) + 1
    
    # memoization array
    memo = np.full(shape=(max_len,max_len), fill_value=-1)
    
    # Get the limits for each melody
    lim_A = len(melody_w_times_A)
    lim_B = len(melody_w_times_B)
    
    # Actual DP implementation
    for i in range(lim_A, -1, -1):
        for j in range(lim_B, -1, -1):
            
            # If we are at the limits the solution is 0
            if i == lim_A or  j == lim_B:
                memo[i][j] = 0
                continue
            
            # If there is a match a possible solution is the previous plus one
            curr_value = 0
            
            tot_delta_time = (float(melody_w_times_A[i][1]) + float(melody_w_times_B[j][1])) / float(DIV_CONST)
            tot_diff_time = np.abs(float(melody_w_times_A[i][1]) - float(melody_w_times_B[j][1]))
            
            
            if (melody_w_times_A[i][3] == melody_w_times_B[j][3]) and (tot_diff_time <= tot_delta_time):
                curr_value = memo[i + 1][j + 1] + 1
                
            # The actual solution is the maximum between the one if there is a match, or skip on the melody A or melody B
            curr_value = max(curr_value, max(memo[i + 1][j], memo[i][j + 1]))
            
            # Save the solution
            memo[i][j] = curr_value
    
    # With the memoization table we can retrieve the actual melody
    i = 0
    j = 0
    SCLM = []
    while i != lim_A and j != lim_B:
    
        if ((memo[i + 1][j + 1] + 1) == memo[i][j]):
            SCLM.append((i, j))
            i += 1
            j += 1
        elif (memo[i + 1][j] == memo[i][j]):
            i += 1
        elif (memo[i][j + 1] == memo[i][j]):
            j += 1
    
    return SCLM

In [7]:
def get_MTRC_v100_from_melody_w_times(melody_w_times_A, melody_w_times_B):
    
    # Assert at least one element for each melody
    if (len(melody_w_times_A) == 0 or len(melody_w_times_B) == 0):
        return 1
    
    # Initialize result variable
    result_value = 0
    
    # Get SCLM v100
    sclm = get_SCLM_v100(melody_w_times_A, melody_w_times_B)
    
    # Get max len
    max_len = max(len(melody_w_times_A), len(melody_w_times_B))
    
    # D3: SCLM Length
    sclmlen_dif3 = (max_len - len(sclm)) / max_len
    result_value += sclmlen_dif3
    
    return result_value

### Local Aligment similarity

In [8]:
def pitch_rater(element_A, element_B):
    if element_A == element_B:
        return 1.0
    else:
        return -1.0

In [9]:
def get_Local_Alignment_from_melody_w_times(melody_w_times_A, melody_w_times_B, insertion_weight=-.5, deletion_weight=-.5, sim_score=pitch_rater):
    # Assert at least one element for each melody
    if (len(melody_w_times_A) == 0 or len(melody_w_times_B) == 0):
        return 1.0
    
    # We use a Dynamic Programming approach
    max_len = max(len(melody_w_times_A), len(melody_w_times_B)) + 1
    
    # memoization array
    memo = np.full(shape=(max_len,max_len), fill_value=-1)
    
    # Get the limits for each melody
    lim_A = len(melody_w_times_A)
    lim_B = len(melody_w_times_B)
    
    max_score = 0.0
    for i in range(lim_A, -1, -1):
        for j in range(lim_B, -1, -1):
            
            # If we are at the limits the solution is 0
            if i == lim_A or  j == lim_B:
                memo[i][j] = 0
                continue
            
            from_left = memo[i][j + 1] + deletion_weight
            from_top = memo[i + 1][j] + insertion_weight
            diag = memo[i + 1][j + 1]  + sim_score(melody_w_times_A[i][2],melody_w_times_B[j][2])
            
            
            memo[i][j] = max(from_top, from_left, diag, 0.0)
            if memo[i][j] > max_score:
                max_score = memo[i][j]
    return -max_score

### Euclidean Distance

In [10]:
def get_pitch_vector(melody_w_times):
    return [float(x[2]) for x in melody_w_times]

In [11]:
def get_Ecuclidean_Distance_from_melody_w_times(melody_w_times_query, melody_w_times_test):
    
    if (len(melody_w_times_query) == 0 or len(melody_w_times_test) == 0):
        return 0
    
    pitch_vector_query = get_pitch_vector(melody_w_times_query)
    pitch_vector_test = get_pitch_vector(melody_w_times_test)
    
    max_len = max(len(pitch_vector_query), len(pitch_vector_test))
    # max_len = float(max(len(pitch_vector_query), len(pitch_vector_test)))
    # min_len = min(len(pitch_vector_query), len(pitch_vector_test))
    
    pitch_vector_query.extend([0] * (max_len - len(pitch_vector_query)))
    pitch_vector_test.extend([0] * (max_len - len(pitch_vector_test)))
    
    sim = spatial.distance.euclidean(pitch_vector_query, pitch_vector_test)
    
    return sim / len(pitch_vector_query)
    # return sim / max_len

### SIAM

In [12]:
def get_SIAM_from_melody_w_times(melody_w_times_query, melody_w_times_test):
    
    if (len(melody_w_times_query) == 0 or len(melody_w_times_test) == 0):
        return 0
    
    start_onset = melody_w_times_query[0][0]
    query_points = [(s[0] - start_onset, s[2]) for s in melody_w_times_query]
    
    translation_vectors = []
    translation_vectors_with_position = []
    
    test_points = np.array([(s[0], s[2]) for s in melody_w_times_test])
    
    for p in query_points:
        vectors = (test_points - p)
        translation_vectors.extend([tuple(v) for v in vectors])
        translation_vectors_with_position.append((p[0], [tuple(v) for v in vectors]))
    
    grouped_vectors = dict(Counter(translation_vectors))
    similarity = max([grouped_vectors[k] for k in grouped_vectors])     
    
    return -(similarity / float(len(query_points)))

### Correlation Distance

In [13]:
def get_Correlation_Distance_from_melody_w_times(melody_w_times_query, melody_w_times_test):
    
    if (len(melody_w_times_query) == 0 or len(melody_w_times_test) == 0):
        return 0
    
    pitch_vector_query = get_pitch_vector(melody_w_times_query)
    pitch_vector_test = get_pitch_vector(melody_w_times_test)
    
    max_len = max(len(pitch_vector_query), len(pitch_vector_test))
    # max_len = float(max(len(pitch_vector_query), len(pitch_vector_test)))
    # min_len = min(len(pitch_vector_query), len(pitch_vector_test))
    
    pitch_vector_query.extend([0] * (max_len - len(pitch_vector_query)))
    pitch_vector_test.extend([0] * (max_len - len(pitch_vector_test)))
    
    sim = spatial.distance.correlation(pitch_vector_query, pitch_vector_test)
    
    return sim / len(pitch_vector_query)
    # return sim / max_len

### City Block Distance

In [14]:
def get_CityBlock_Distance_from_melody_w_times(melody_w_times_query, melody_w_times_test):
    
    if (len(melody_w_times_query) == 0 or len(melody_w_times_test) == 0):
        return 0
    
    pitch_vector_query = get_pitch_vector(melody_w_times_query)
    pitch_vector_test = get_pitch_vector(melody_w_times_test)
    
    max_len = max(len(pitch_vector_query), len(pitch_vector_test))
    # max_len = float(max(len(pitch_vector_query), len(pitch_vector_test)))
    # min_len = min(len(pitch_vector_query), len(pitch_vector_test))
    
    pitch_vector_query.extend([0] * (max_len - len(pitch_vector_query)))
    pitch_vector_test.extend([0] * (max_len - len(pitch_vector_test)))
    
    sim = spatial.distance.cityblock(pitch_vector_query, pitch_vector_test)
    
    return sim / len(pitch_vector_query)
    # return sim / max_len

### Control for similarity measure to use

In [15]:
def similarity_evaluation(measure_key, melody_w_times_A, melody_w_times_B):
    
    if measure_key == "AlRyNo":
        return get_MTRC_v100_from_melody_w_times(melody_w_times_A, melody_w_times_B)
    if measure_key == "LA":
        return get_Local_Alignment_from_melody_w_times(melody_w_times_A, melody_w_times_B)
    if measure_key == "ED":
        return get_Ecuclidean_Distance_from_melody_w_times(melody_w_times_A, melody_w_times_B)
    if measure_key == "SIAM":
        return get_SIAM_from_melody_w_times(melody_w_times_A, melody_w_times_B)
    if measure_key == "CORR":
        return get_Correlation_Distance_from_melody_w_times(melody_w_times_A, melody_w_times_B)
    if measure_key == "CBD":
        return get_CityBlock_Distance_from_melody_w_times(melody_w_times_A, melody_w_times_B)
    
    return 1.0

# Groups for analysis
 * Reference for the axis
 * Groups of dataset

In [16]:
Y_AXIS_MIDI_PATHS = [
    "./CalebRascon/CORPUS/MIDI/",
    "./CalebRascon/MIDI_Grammar_SOLO_LEN12/",
    "./MilesDavis/MIDI_Grammar_SOLO_LEN12/",
    "./CharlieParker/MIDI_Grammar_SOLO_LEN12/"
]

In [17]:
X_AXIS_MIDI_PATHS = [
    "./CalebRascon/CORPUS/MIDI/",
    "./CalebRascon/MIDI_Grammar_SOLO_LEN12/",
    "./MilesDavis/MIDI_Grammar_SOLO_LEN12/",
    "./CharlieParker/MIDI_Grammar_SOLO_LEN12/",
    "./CalebRascon/MIDI_Grammar_TRADE_Caleb/",
    "./CalebRascon/MIDI_Grammar_TRADE_Miles/",
    "./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/"
]

In [18]:
SIMILARITY_MEASURE_KEY = "AlRyNo"
# SIMILARITY_MEASURE_KEY = "LA"
# SIMILARITY_MEASURE_KEY = "ED"
# SIMILARITY_MEASURE_KEY = "SIAM"
# SIMILARITY_MEASURE_KEY = "CORR"
# SIMILARITY_MEASURE_KEY = "CBD"

In [19]:
%%time
all_similarities_store = {}
for MIDI_path_query in Y_AXIS_MIDI_PATHS:
    for MIDI_path_test in X_AXIS_MIDI_PATHS:

        similarities_from_references = []
        similarities_all_v_all = {}
        for root_ref, dirs_ref, files_ref in os.walk(MIDI_path_query):
            for name_ref in files_ref:
                # print("+++++++++++++++++++++++++++++")
                # print(name_ref)
                melody_score_A = m21.converter.parseFile(os.path.join(root_ref, name_ref))
                midi_tracks_A = m21.midi.translate.streamToMidiFile(melody_score_A)
                melody_w_times_A = getMelodyDeltaTimes(midi_tracks_A.tracks[0].events)

                similarities_from_reference = []
                similarities_all_v_all[name_ref] = {}
                for root, dirs, files in os.walk(MIDI_path_test):
                    for name in files:

                        # print(name)
                        melody_score_B = m21.converter.parseFile(os.path.join(root, name))
                        midi_tracks_B = m21.midi.translate.streamToMidiFile(melody_score_B)
                        melody_w_times_B = getMelodyDeltaTimes(midi_tracks_B.tracks[0].events)
                        
                        if (name == name_ref) and (root == root_ref):
                            current_similarity = 0.0;
                        else:
                            current_similarity = similarity_evaluation(
                                SIMILARITY_MEASURE_KEY, melody_w_times_A, melody_w_times_B)

                        similarities_from_reference.append(current_similarity)
                        similarities_all_v_all[name_ref][name] = current_similarity
                        # print(current_similarity)
                similarities_from_references.append(similarities_from_reference)
        similarities_df = pd.DataFrame(similarities_all_v_all)
        similarities_df = similarities_df.reindex(sorted(similarities_df.columns), axis=1)
        similarities_df = similarities_df.sort_index()
        similarities_df = similarities_df.replace(0, np.NaN)
        display(similarities_df)
        print(similarities_df.mean(axis=1))
        keyname_sim = MIDI_path_query+":"+MIDI_path_test
        print(keyname_sim)
        all_similarities_store[keyname_sim] = similarities_df.to_dict('index')

,rola1.mid,rola2.mid,rola3.mid,rola4.mid,rola5.mid
rola1.mid,NaN,0.796610,0.627119,0.677966,0.593220
rola2.mid,0.796610,NaN,0.780488,0.763636,0.769231
rola3.mid,0.627119,0.780488,NaN,0.636364,0.653846
rola4.mid,0.677966,0.763636,0.636364,NaN,0.563636
rola5.mid,0.593220,0.769231,0.653846,0.563636,NaN


rola1.mid    0.673729
rola2.mid    0.777491
rola3.mid    0.674454
rola4.mid    0.660401
rola5.mid    0.644983
dtype: float64
./CalebRascon/CORPUS/MIDI/:./CalebRascon/CORPUS/MIDI/


,rola1.mid,rola2.mid,rola3.mid,rola4.mid,rola5.mid
CalebRascon_Grammar_SOLO_LEN12_001.mid,0.779661,0.810811,0.756098,0.836364,0.788462
CalebRascon_Grammar_SOLO_LEN12_002.mid,0.830508,0.853659,0.804878,0.872727,0.846154
CalebRascon_Grammar_SOLO_LEN12_003.mid,0.762712,0.851064,0.765957,0.781818,0.750000
CalebRascon_Grammar_SOLO_LEN12_004.mid,0.898305,0.857143,0.878049,0.890909,0.884615
CalebRascon_Grammar_SOLO_LEN12_005.mid,0.915254,0.857143,0.902439,0.890909,0.942308


CalebRascon_Grammar_SOLO_LEN12_001.mid    0.794279
CalebRascon_Grammar_SOLO_LEN12_002.mid    0.841585
CalebRascon_Grammar_SOLO_LEN12_003.mid    0.782310
CalebRascon_Grammar_SOLO_LEN12_004.mid    0.881804
CalebRascon_Grammar_SOLO_LEN12_005.mid    0.901611
dtype: float64
./CalebRascon/CORPUS/MIDI/:./CalebRascon/MIDI_Grammar_SOLO_LEN12/


,rola1.mid,rola2.mid,rola3.mid,rola4.mid,rola5.mid
MilesDavis_Grammar_SOLO_LEN12_001.mid,0.830508,0.714286,0.756098,0.781818,0.788462
MilesDavis_Grammar_SOLO_LEN12_002.mid,0.745763,0.744186,0.767442,0.745455,0.769231
MilesDavis_Grammar_SOLO_LEN12_003.mid,0.762712,0.795455,0.772727,0.763636,0.750000
MilesDavis_Grammar_SOLO_LEN12_004.mid,0.779661,0.756098,0.756098,0.763636,0.769231
MilesDavis_Grammar_SOLO_LEN12_005.mid,0.728814,0.736842,0.804878,0.745455,0.807692


...

Series([], dtype: float64)
./CharlieParker/MIDI_Grammar_SOLO_LEN12/:./CalebRascon/MIDI_Grammar_TRADE_Caleb/


,CharlieParker_Grammar_SOLO_LEN12_001.mid,CharlieParker_Grammar_SOLO_LEN12_002.mid,CharlieParker_Grammar_SOLO_LEN12_003.mid,CharlieParker_Grammar_SOLO_LEN12_004.mid,CharlieParker_Grammar_SOLO_LEN12_005.mid
trade_Caleb_Miles_rola1.mid,0.880952,0.821429,0.836957,0.728814,0.828571
trade_Caleb_Miles_rola2.mid,0.809524,0.773810,0.782609,0.644068,0.785714
trade_Caleb_Miles_rola3.mid,0.904762,0.892857,0.913043,0.864407,0.900000
trade_Caleb_Miles_rola4.mid,0.857143,0.857143,0.880435,0.762712,0.857143
trade_Caleb_Miles_rola5.mid,0.809524,0.738095,0.793478,0.644068,0.714286


trade_Caleb_Miles_rola1.mid    0.819344
trade_Caleb_Miles_rola2.mid    0.759145
trade_Caleb_Miles_rola3.mid    0.895014
trade_Caleb_Miles_rola4.mid    0.842915
trade_Caleb_Miles_rola5.mid    0.739890
dtype: float64
./CharlieParker/MIDI_Grammar_SOLO_LEN12/:./CalebRascon/MIDI_Grammar_TRADE_Miles/


,CharlieParker_Grammar_SOLO_LEN12_001.mid,CharlieParker_Grammar_SOLO_LEN12_002.mid,CharlieParker_Grammar_SOLO_LEN12_003.mid,CharlieParker_Grammar_SOLO_LEN12_004.mid,CharlieParker_Grammar_SOLO_LEN12_005.mid
trade_Caleb_CharlieParker_rola1.mid,0.726190,0.619048,0.673913,0.654321,0.703704
trade_Caleb_CharlieParker_rola2.mid,0.750000,0.702381,0.750000,0.600000,0.685714
trade_Caleb_CharlieParker_rola3.mid,0.750000,0.690476,0.739130,0.628571,0.728571
trade_Caleb_CharlieParker_rola4.mid,0.690476,0.630952,0.717391,0.629630,0.666667
trade_Caleb_CharlieParker_rola5.mid,0.821429,0.714286,0.739130,0.627119,0.728571


trade_Caleb_CharlieParker_rola1.mid    0.675435
trade_Caleb_CharlieParker_rola2.mid    0.697619
trade_Caleb_CharlieParker_rola3.mid    0.707350
trade_Caleb_CharlieParker_rola4.mid    0.667023
trade_Caleb_CharlieParker_rola5.mid    0.726107
dtype: float64
./CharlieParker/MIDI_Grammar_SOLO_LEN12/:./CalebRascon/MIDI_Grammar_TRADE_CharlieParker/
CPU times: user 1min 36s, sys: 50.4 ms, total: 1min 36s
Wall time: 1min 36s


In [20]:
with open('./CCEvaluation_groups.json', 'w') as outfile:
    json.dump(all_similarities_store, outfile)